In [193]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels
using LaTeXTabulars

In [194]:
Random.seed!(02102023);

# Simulate data
Let $\eta_t \sim N(0,1)$ i.i.d.; $\epsilon_{it} \sim N(0,1)$ i.i.d.; $x_t \sim \exp(N(0,1))$ i.i.d.; $z_{it} \sim N(0,2)$ i.i.d.

Let $K=30$ and $T=100$

True parameters:

- $\alpha = 1$
- $\beta = 2$ 
- $\delta = 6$ 
- $\gamma = 3$ 
- $\rho = 0.8$

In [195]:
# parameters 
α = 1; 
β = 2;
δ = 6;
γ = 3; 
ρ = 0.8;

# markets and firms
K = 30; 
T = 100; 

# specify distributions 
normal_0_1 = Normal(0,1);
lognormal_0_1 = LogNormal(0,1);
normal_0_2 = Normal(0, 2);

In [196]:
# market level 
market_df = DataFrame(market = 1:T,
    ηₜ = rand(normal_0_1, T), 
    xₜ = rand(lognormal_0_1, T));

# firm-market level
firm_market_df = DataFrame(firm = repeat(1:K, inner=T),
    market = repeat(1:T, outer=K),
    ϵᵢₜ = rand(normal_0_1, K * T),
    zᵢₜ = rand(normal_0_2, K * T));
firm_market_df = innerjoin(firm_market_df, market_df, on=:market);
firm_market_df[!,:ϕᵢₜ] = firm_market_df[!,:zᵢₜ] .* α .+ ρ .* firm_market_df[!,:ηₜ] .+ sqrt(1-ρ^2) .* firm_market_df[!,:ϵᵢₜ]

# sort by heterogenous profitability in a market
sort!(firm_market_df, [:market, order(:ϕᵢₜ, rev=true)]);
firm_market_df[!,:nₜ] = repeat(1:K, T); #this gives number of firms by sequential entry 
firm_market_df[!,:πᵢₜ] = γ .+ β .* firm_market_df[!,:xₜ] .- δ .* log.(firm_market_df[!,:nₜ]) .+ firm_market_df[!,:ϕᵢₜ];
firm_market_df[!,:Iᵢₜ] = ifelse.(firm_market_df[!,:πᵢₜ] .>=0, 1, 0)
firm_market_df = transform(groupby(firm_market_df, :market), :Iᵢₜ => sum => :nₜ);


In [197]:
sort!(firm_market_df, [:market, :firm]);

# Estimating via Simulation

In [198]:
# function that draws shocks
get_shock = function(S, input_df)
    shocks = DataFrame(s_draw = Any[], firm = Any[], market = Any[], epsilon = Any[], eta = Any[], zᵢₜ = Any[], xₜ = Any[])
    for s in 1:S
        eta = DataFrame(market = repeat(1:T),
            eta = rand(normal_0_1, T))
        epsilon = DataFrame(firm = repeat(1:K, inner=T),
            market = repeat(1:T, outer=K),
            epsilon = rand(normal_0_1, K * T))
        df = innerjoin(epsilon, eta, on=:market)
        df[!,:s_draw] = repeat([s], K*T)

        df = innerjoin(df, firm_market_df, on=[:firm, :market])
        select!(df, [:s_draw, :firm, :market, :epsilon, :eta, :zᵢₜ, :xₜ])
        
        append!(shocks, df)
    end
    return shocks
end

# function that calculates SPNEs given simulated df 
sim_spne = function(α, β, δ, γ, ρ, S, K, T, sim_df, actual_df)
    
    sim_df[!,:ϕᵢₜ] = sim_df[!,:zᵢₜ] .* α .+ ρ .* sim_df[!,:eta] .+ sqrt(1-ρ^2) .* sim_df[!,:epsilon]
    sort!(sim_df, [:s_draw, :market, order(:ϕᵢₜ, rev=true)]);
    sim_df[!,:nₜ] = repeat(1:K, S * T);
    sim_df[!,:πᵢₜ] = γ .+ β .* sim_df[!,:xₜ] .- δ .* log.(sim_df[!,:nₜ]) .+ sim_df[!,:ϕᵢₜ];
    sim_df[!,:Iᵢₜ] = ifelse.(sim_df[!,:πᵢₜ] .>=0, 1, 0)
    sim_df = transform(groupby(sim_df, [:s_draw, :market]), :Iᵢₜ => sum => :nₜ)
    sim_df = select(sim_df, [:firm, :market, :Iᵢₜ, :nₜ])
    sim_df = groupby(sim_df, [:firm, :market])
    sim_df = combine(sim_df, :Iᵢₜ => mean => :predIᵢₜ)
    
    df = innerjoin(sim_df, actual_df, on=[:firm, :market])
    
    return df
end

# gmm objective 
odell_beckham_jr = function(K, T, input_df)
    
    # moment condition for each firm i = 1...K
    moments = zeros(K)
    for i in 1:K
        firm = filter(:firm => n -> n .== i, input_df)    
        moments[i] = transpose(firm[!,:Iᵢₜ] .- firm[!,:predIᵢₜ]) * firm[!,:zᵢₜ]
    end
    
    # weighting matrix is just identity for now
    obj = moments' * I * moments
    return obj 
end

# optim wrapper
get_argmax = function(θ, sim_df, actual_df)
    α = θ[1]
    β = θ[2]
    δ = θ[3]
    γ = θ[4] 
    ρ = θ[5]
    S = 100
    K = 30
    T = 100
    
    df = sim_spne(α, β, δ, γ, ρ, S, K, T, sim_df, actual_df)
    obj = odell_beckham_jr(K, T, df)
end


#110 (generic function with 1 method)

In [199]:
# plot objective over all possible values of alpha 
alpha = zeros(length(range(-5, 5, step=1)))
iter = 1
shocks = get_shock(100, firm_market_df)

for a in range(-5, 5, step=1)
    sim_data = sim_spne(a, β, δ, γ, ρ, 100, K, T, shocks, firm_market_df)
    alpha[iter] = odell_beckham_jr(K, T, sim_data)
    iter += 1
end

In [200]:
plot(-5:5, alpha, legend = false)
savefig("output/ps2_q2_gmm_obj.pdf")

"/Users/junwong/Documents/io2_psets/ps2/output/ps2_q2_gmm_obj.pdf"

In [201]:
repeat_estimation = function(num, params0)
    alpha = zeros(num)
    beta = zeros(num)
    delta = zeros(num)
    gamma = zeros(num)
    rho = zeros(num)
    for i in 1:num
        shocks = get_shock(100, firm_market_df)
        f(θ) = get_argmax(θ, shocks, firm_market_df)
        #params0 = [1.0 ; 2.0 ; 6.5; 1.0; 0.5]
        o = Optim.optimize(f, params0, NelderMead(), Optim.Options(show_trace = true, show_every = 100))

        alpha[i] = o.minimizer[1]
        beta[i] = o.minimizer[2]
        delta[i] = o.minimizer[3]
        gamma[i] = o.minimizer[4]
        rho[i] = o.minimizer[5]
    end
    
    return alpha, beta, delta, gamma, rho
end

#112 (generic function with 1 method)

In [202]:
alpha, beta, delta, gamma, rho = repeat_estimation(10, [1.0 ; 2.0 ; 6.5; 1.0; 0.5])
histogram(alpha, bins=5,legend=false)
savefig("output/ps2_q2_alpha.pdf")
histogram(beta, bins=5, legend=false)
savefig("output/ps2_q2_beta.pdf")
histogram(gamma, bins=5, legend=false)
savefig("output/ps2_q2_gamma.pdf")
histogram(delta, bins=5, legend=false)
savefig("output/ps2_q2_delta.pdf")
histogram(rho, bins=5, legend=false)
savefig("output/ps2_q2_rho.pdf")

alpha, beta, delta, gamma, rho = repeat_estimation(10, [2.0 ; 0.0 ; 1.0; 0.0; 0.3])
histogram(alpha, bins=5,legend=false)
savefig("output/ps2_q2_alpha_alt.pdf")
histogram(beta, bins=5, legend=false)
savefig("output/ps2_q2_beta_alt.pdf")
histogram(gamma, bins=5, legend=false)
savefig("output/ps2_q2_gamma_alt.pdf")
histogram(delta, bins=5, legend=false)
savefig("output/ps2_q2_delta_alt.pdf")
histogram(rho, bins=5, legend=false)
savefig("output/ps2_q2_rho_alt.pdf")


Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     9.817819e+02     2.819168e+03
 * time: 0.0001361370086669922
   100     5.698554e+02     7.514156e-02
 * time: 60.75241804122925
Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     9.587287e+02     2.810495e+03
 * time: 7.200241088867188e-5
   100     5.222421e+02     0.000000e+00
 * time: 60.66123104095459
Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     9.905943e+02     2.781066e+03
 * time: 6.318092346191406e-5
   100     6.047379e+02     1.817361e-01
 * time: 55.7504141330719
Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     1.027256e+03     2.807047e+03
 * time: 6.198883056640625e-5
   100     6.067566e+02     2.252121e-01
 * time: 58.66519594192505
Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0     9.849024

"/Users/junwong/Documents/io2_psets/ps2/output/ps2_q2_rho_alt.pdf"